In [1]:
import pandas as pd
import sys

In [3]:
sys.path.append('..')

In [4]:
from scripts import util

In [5]:
conn = util.connect_rds()

The aim of this notebook is to explore the following questions:
 - [ ] Does CSR ongevellan have similar numbers as the incident data that has been provided by RWS
 - [ ] Is there a common key between the 2 datasets such that we can beef up RWS using Ongavellen.

In [44]:
rws = pd.read_sql('select * from rws_schema.ongevallen_raw;', con=conn)

In [9]:
csr = pd.read_sql('select * from rws_schema.incidents;', con=conn)

In [10]:
csr.head()

,id,road_name,road_chr,road_ind,road_part_long,hectometer,lat,long,deployment_region,deployment_area,...,inc_start,inc_end,inc_type,num_officers,inc_deploy,inc_arrive,inc_end_off,geom,coord_hecto_id,inc_hect_dist
0,"2,396,988",A50,A,50,Re,170.2,52.0259236574,5.821207004,NA,NA,...,2014-01-05 12:51:34,2014-01-05 13:06:46,Ongeval,1,2014-01-05 12:51:52,2014-01-05 13:05:24,2014-01-05 13:05:24,0101000020E610000000000080EA481740000000805103...,"125,530",0.11403182
1,"2,397,750",N325,N,325,Re,26.3,51.9739076149,5.9585225048,NA,NA,...,2014-01-07 17:33:14,2014-01-07 18:02:29,Ongeval,1,2014-01-07 17:33:51,2014-01-07 17:33:52,2014-01-07 18:01:27,0101000020E6100000000000E086D5174000000000A9FC...,"75,301",0.02043328
2,"2,400,730",N276,N,276,Li,12.7,50.9808897548,5.8663595092,NA,NA,...,2014-01-17 11:15:27,2014-01-17 11:34:23,Pechgeval,1,2014-01-17 11:17:17,2014-01-17 11:17:30,2014-01-17 11:34:12,0101000020E61000000000000027771740000000C08D7D...,"43,967",0.15499474
3,"2,942,248",N201,N,201,Li,36,52.28341,4.774547,NA,NA,...,2017-10-12 09:08:20,2017-10-12 09:12:25,Pechgeval,1,2017-10-12 09:09:15,2017-10-12 09:09:22,2017-10-12 09:11:02,0101000020E6100000000000E022191340000000C04624...,"114,793",0.1814561
4,"2,403,652",A58,A,58,Re,94.2,51.5299855375,4.4322033411,NA,NA,...,2014-01-27 17:25:03,2014-01-27 17:28:38,Ongeval,1,2014-01-27 17:25:33,2014-01-27 17:25:35,2014-01-27 17:28:37,0101000020E61000000000008093BA1140000000A0D6C3...,"65,567",0.1999231


In [11]:
rws.columns

Index(['id', 'source', 'datetime', 'direction', 'x', 'y', 'ending', 'weg',
       'key', 'hecto', 'hectomtrng', 'dvk_letter', 'pos_tv_wol', 'hecto_key'],
      dtype='object')

In [12]:
csr.columns

Index(['id', 'road_name', 'road_chr', 'road_ind', 'road_part_long',
       'hectometer', 'lat', 'long', 'deployment_region', 'deployment_area',
       'inc_id', 'inc_start', 'inc_end', 'inc_type', 'num_officers',
       'inc_deploy', 'inc_arrive', 'inc_end_off', 'geom', 'coord_hecto_id',
       'inc_hect_dist'],
      dtype='object')

In [24]:
csr.inc_type.value_counts(normalize=True)

Pechgeval                      0.453257
Ongeval                        0.194400
Voorwerp                       0.136411
Dieren                         0.044453
Voorwerp op rijstrook          0.037199
Overige                        0.028501
Handhaving                     0.017654
Schade aan infra               0.016823
Achtergelaten Voertuig         0.013600
Verontreiniging                0.013524
Hoogtemelding                  0.009953
Voorwerp op de vluchtstrook    0.008359
Onwelwording                   0.006127
Brandend voertuig              0.004457
Werk in uitvoering             0.004156
Storing                        0.004024
Brand                          0.003293
Verkeersaanbod                 0.002366
Spookrijder                    0.000474
Regelscenario                  0.000445
Gladheidsbestrijding           0.000418
Flitsprotocol                  0.000067
Shiftjournaal                  0.000020
Weer                           0.000009
Cross Border Management        0.000009


In [32]:
csr.loc[:,'inc_start'] = pd.to_datetime(csr.inc_start)


In [36]:
csr.loc[:,'date'] = csr.inc_start.apply(lambda x: x.date())
csr.loc[:,'year'] = csr.inc_start.apply(lambda x: x.year)

In [39]:
csr.loc[:,'accident'] = 1

In [50]:
d = csr.pivot_table(index='inc_type',columns='year', values='accident', aggfunc=sum)

In [59]:
d.loc['Ongeval',:]

year
2013       53.0
2014    12536.0
2015    16470.0
2016    20563.0
2017    16846.0
Name: Ongeval, dtype: float64

In [60]:
rws.jaar.value_counts().sort_index()

2012    12474
2013    14789
2014    11583
2015    21186
2016    52723
2017    57387
2018     5970
Name: jaar, dtype: int64

In [37]:
csr.head(1).transpose()

,0
id,"2,396,988"
road_name,A50
road_chr,A
road_ind,50
road_part_long,Re
hectometer,170.2
lat,52.0259236574
long,5.821207004
deployment_region,NA
deployment_area,NA


## Do they have a common key?

In [61]:
# what are the common columns

In [65]:
c = set(csr.columns)
r = set(rws.columns)

In [66]:
c.intersection(r)

set()

In [69]:
r.intersection(c)

set()

Because column names have been edited in english, so there is no direct intersection

In [74]:
csr.loc[csr.inc_type=='Ongeval']

,id,road_name,road_chr,road_ind,road_part_long,hectometer,lat,long,deployment_region,deployment_area,...,num_officers,inc_deploy,inc_arrive,inc_end_off,geom,coord_hecto_id,inc_hect_dist,date,year,accident
0,"2,396,988",A50,A,50,Re,170.2,52.0259236574,5.821207004,NA,NA,...,1,2014-01-05 12:51:52,2014-01-05 13:05:24,2014-01-05 13:05:24,0101000020E610000000000080EA481740000000805103...,"125,530",0.11403182,2014-01-05,2014,1
1,"2,397,750",N325,N,325,Re,26.3,51.9739076149,5.9585225048,NA,NA,...,1,2014-01-07 17:33:51,2014-01-07 17:33:52,2014-01-07 18:01:27,0101000020E6100000000000E086D5174000000000A9FC...,"75,301",0.02043328,2014-01-07,2014,1
4,"2,403,652",A58,A,58,Re,94.2,51.5299855375,4.4322033411,NA,NA,...,1,2014-01-27 17:25:33,2014-01-27 17:25:35,2014-01-27 17:28:37,0101000020E61000000000008093BA1140000000A0D6C3...,"65,567",0.1999231,2014-01-27,2014,1
5,"2,404,036",N325,N,325,Li,24.7,51.9619630099,5.9462904147,NA,NA,...,1,2014-01-29 04:01:44,2014-01-29 04:05:31,2014-01-29 04:26:14,0101000020E61000000000006000C91740000000A021FB...,"22,729",0.07176929,2014-01-29,2014,1
6,"2,964,994",N302,N,302,Re,105.2,52.36159,5.616934,NA,NA,...,1,2017-09-11 16:35:32,2017-09-11 16:35:34,2017-09-11 16:36:14,0101000020E610000000000080BD771640000000A0482E...,"34,007",0.69933248,2017-09-11,2017,1
11,"2,412,382",A31,A,31,Li,19.7,53.19303,5.488835,Noord-Oost-Nederland,208,...,1,2014-02-26 06:07:51,2014-02-26 06:25:32,2014-02-26 06:25:33,0101000020E61000000000002091F4154000000040B598...,"111,027",0.37198527,2014-02-26,2014,1
13,"2,413,082",A58,A,58,Li,93.8,51.52959,4.437923,NA,NA,...,2,2014-02-27 21:32:37,2014-02-27 21:39:31,2014-02-27 23:08:26,0101000020E6100000000000E06EC01140000000A0C9C3...,"156,883",0.13635965,2014-02-27,2014,1
24,"2,422,561",A20,A,20,Li,31.4,51.94048,4.470038,Randstad Zuid,107,...,2,2014-04-01 16:24:23,2014-04-01 16:35:00,2014-04-01 17:33:12,0101000020E6100000000000A051E11140000000A061F8...,"147,146",0.09716698,2014-04-01,2014,1
25,"2,991,205",A4,A,4,Li,94.6,51.7348,4.41094,NA,NA,...,1,2015-07-08 09:39:05,2015-07-08 10:09:47,2015-07-08 10:20:11,0101000020E610000000000080CDA41140000000E00DDE...,"134,989",0.35129226,2015-07-08,2015,1
32,"2,432,452",N229,N,229,Li,12.9,52.01161,5.271406,NA,NA,...,1,2014-05-03 09:51:13,2014-05-03 10:24:29,2014-05-03 10:24:30,0101000020E610000000000080EB151540000000607C01...,"122,551",0.65510358,2014-05-03,2014,1


In [72]:
rws.head()

,id_jaar,ongeval_id,bron,datumtijd,uur,dag,maand,jaar,bps_key,wvkid_begdat_hecto_key,rijrichting,coords_x,coords_y,afloop,key,weg,hecto,hectomtrng,dvk_letter,pos_tv_wol
0,90088.2018,90088,NDW,2018-02-19 14:56:30,14,19,2,2018,None,497347012.2017-11-01.608,None,"6,75935506820679","52,2414283752441",None,35.608.Li,35,"60,8",608,None,Li
1,90087.2018,90087,NDW,2018-02-19 14:25:46,14,19,2,2018,None,275279004.2017-09-01.795,None,"5,14758014678955","51,9416618347168",None,2.795.Li,2,"79,5",795,None,Li
2,90086.2018,90086,NDW,2018-02-16 06:31:28,6,16,2,2018,None,177303013.2017-12-01.99,None,"4,42396306991577","52,0517883300781",None,12.99.Re,12,"9,9",99,None,Re
3,90085.2018,90085,NDW,2018-02-15 15:11:22,15,15,2,2018,None,196267066.2014-11-01.236,None,"4,56472015380859","51,8843116760254",None,16.236.y,16,"23,6",236,y,Li
4,90084.2018,90084,NDW,2018-02-15 10:51:55,10,15,2,2018,None,216377018.2015-04-01.97,None,"4,70029401779175","52,3842010498047",None,200.97.c,200,"9,7",97,c,Li


In [83]:
csr.shape

(341914, 24)

In [82]:
pd.to_numeric(rws.id_jaar.map(lambda x: x.split('.')[0])).describe()

count    176112.000000
mean      88056.500000
std       50839.299641
min           1.000000
25%       44028.750000
50%       88056.500000
75%      132084.250000
max      176112.000000
Name: id_jaar, dtype: float64